<a href="https://colab.research.google.com/github/rickielin/Cardiac_Segmentation/blob/main/Cardiac_2_Cine_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pydicom SimpleITK dicom2nifti

import numpy as np
import pandas as pd
from numpy import *
import seaborn as sns
import scipy.io as scio
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import os
import pydicom
import random
import glob
from tqdm import tqdm
import SimpleITK as sitk
import cv2
import nibabel as nib
import dicom2nifti
from skimage.transform import resize
%matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
DATASET_FOLDER = "/content/drive/MyDrive/Temporary/Cine_LGE_samples"
# 5 example cases only

In [7]:
patient_case_num = []
for i in os.listdir(DATASET_FOLDER):
    patient_case_num.append(i)

patient_case_num

['4', '49', '26', '83', '310', 'Matching_frame_number_in_Cine_images.xlsx']

In [9]:
patient_case_num.remove('Matching_frame_number_in_Cine_images.xlsx')
patient_case_num

['4', '49', '26', '83', '310']

Cine images will use the shared annotation files used by LGE.

So this notebook only aims at converting Cine from Dicom slices to NIFTI files.

In [12]:
# Matching_frame_number_in_Cine_images - TABLE

df_match = pd.read_excel('/content/drive/MyDrive/Temporary/Cine_LGE_samples/Matching_frame_number_in_Cine_images.xlsx')
print(df_match.head())

   CaseID  Matching frame number in Cine images
0       4                                    22
1       6                                    22
2      17                                    20
3      18                                    21
4      21                                    22


In [21]:
filtered_df = df_match[df_match['CaseID'] == 17]
print(filtered_df['Matching frame number in Cine images'].values[0])

20


In [11]:
!mkdir CINE_NII_FOLDER

In [26]:
import glob
files = glob.glob('/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new' + '/*')

files = sorted(files)  # 对文件名进行排序

# for file in files:
#     print(file)
match_num = df_match[df_match['CaseID'] == 4]['Matching frame number in Cine images'].values[0]
print(files[match_num - 1])
files[21] # 22-1=21

/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00142.dcm


'/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00142.dcm'

In [44]:
# 把annotation files也 resize 到(512, 512, 1)

nii = nib.load('/content/49_3.nii.gz')
data = nii.get_fdata()
resized_data = resize(data, (512, 512, 1))
new_nii = nib.Nifti1Image(resized_data, np.eye(4))
nib.save(new_nii, '/content/49_3_label.nii.gz')

In [43]:
# DICOM 文件路径
dicom_file = "/content/drive/MyDrive/Temporary/Cine_LGE_samples/49/49_3/Cine_new/IMG-0009-00139.dcm"

# 输出 NIfTI 文件路径
nifti_file = "/content/49_3_cine.nii.gz"

# testing: Dicom --> NIFTI
from skimage.transform import resize

dcm = pydicom.dcmread(dicom_file) # 读取 DICOM 文件
pixel_array = dcm.pixel_array # 获取像素数据
pixel_array_3d = pixel_array[:,:,np.newaxis] # 把2D数组转换为3D
# pixel_array_3d = np.expand_dims(pixel_array, axis=0) # 把2D数组转换为3D

resized_pixel_array = resize(pixel_array_3d, (512, 512, 1)) # 改变图像大小到(512, 512, 1)
flipped_pixel_array = np.flip(resized_pixel_array, axis=0) # 上下翻折图像
rotated_pixel_array = np.rot90(flipped_pixel_array, 3, (0, 1)) # 逆时针旋转270度

nii = nib.Nifti1Image(rotated_pixel_array, np.eye(4)) # 创建 NIfTI 图像
nib.save(nii, nifti_file)

## 最终代码

In [48]:
# 大循环1_生成所有nii格式的Cine

# Path to store NIFTI files
CINE_NII = '/content/CINE_NII_FOLDER'

for case_num in patient_case_num:
    # 获取 _n 的个数
    Infarct_mask = scio.loadmat(DATASET_FOLDER+f"/{case_num}/Masks/InfarctMask.mat")
    Infarct_array = Infarct_mask['InfarctMask']
    mri_count = Infarct_array.shape[2]

    for _n in range(mri_count):
        cine_processing = f'{DATASET_FOLDER}/{case_num}/{case_num}_{_n+1}/Cine_new'
        files = glob.glob(cine_processing + '/*')
        files = sorted(files)  # 对文件名进行排序
        match_num = df_match[df_match['CaseID']==int(case_num)]['Matching frame number in Cine images'].values[0] -1
        print(f"processing: {case_num}_{_n+1}, match number == {match_num + 1}")

        cine_processing = f'{files[match_num]}'
        print(cine_processing)

        dicom_file = cine_processing
        nifti_file = f'{CINE_NII}/{case_num}_{_n+1}_0000.nii.gz' # output path

        dcm = pydicom.dcmread(dicom_file) # 读取 DICOM 文件
        pixel_array = dcm.pixel_array # 获取像素数据
        pixel_array_3d = pixel_array[:,:,np.newaxis] # 把2D数组转换为3D
        resized_pixel_array = resize(pixel_array_3d, (512, 512, 1)) # 改变图像大小到(512, 512, 1)
        flipped_pixel_array = np.flip(resized_pixel_array, axis=0) # 上下翻折图像
        rotated_pixel_array = np.rot90(flipped_pixel_array, 3, (0, 1)) # 逆时针旋转270度

        nii = nib.Nifti1Image(rotated_pixel_array, np.eye(4)) # 创建 NIfTI 图像
        nib.save(nii, nifti_file)

processing: 4_1, match number == 22
/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_1/Cine_new/IMG-0001-00142.dcm
processing: 4_2, match number == 22
/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_2/Cine_new/IMG-0001-00112.dcm
processing: 4_3, match number == 22
/content/drive/MyDrive/Temporary/Cine_LGE_samples/4/4_3/Cine_new/IMG-0001-00082.dcm
processing: 49_1, match number == 19
/content/drive/MyDrive/Temporary/Cine_LGE_samples/49/49_1/Cine_new/IMG-0009-00199.dcm
processing: 49_2, match number == 19
/content/drive/MyDrive/Temporary/Cine_LGE_samples/49/49_2/Cine_new/IMG-0009-00169.dcm
processing: 49_3, match number == 19
/content/drive/MyDrive/Temporary/Cine_LGE_samples/49/49_3/Cine_new/IMG-0009-00139.dcm
processing: 49_4, match number == 19
/content/drive/MyDrive/Temporary/Cine_LGE_samples/49/49_4/Cine_new/IMG-0009-00109.dcm
processing: 49_5, match number == 19
/content/drive/MyDrive/Temporary/Cine_LGE_samples/49/49_5/Cine_new/IMG-0009-00079.dcm
processing: 26_1, match n

In [47]:
# 大循环2_将所有annotation files resize 到 (512, 512, 1)

for i in os.listdir("/Users/rickielim/Desktop/Task300_LGE_only"):
    nii = nib.load("/content/sample_data/" + i)
    data = nii.get_fdata()
    resized_data = resize(data, (512, 512, 1))
    new_nii = nib.Nifti1Image(resized_data, np.eye(4))
    nib.save(new_nii, '/content/49_3_label.nii.gz')  # 存301 dataset的labelsTr里！

/content/sample_data/anscombe.json
/content/sample_data/README.md
/content/sample_data/california_housing_train.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/mnist_test.csv
/content/sample_data/mnist_train_small.csv
